In [1]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import OneHotEncoder
import pickle
import pandas as pd
import os
import csv
import numpy as np

/home/sam/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_dir = "../images/Train/"
validation_dir = "../images/Validation"
test_dir = "../images/Test"

In [3]:
a = pd.read_csv("../train.csv")

In [4]:
a.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [5]:
train_generator = train_datagen.flow_from_dataframe(
    a,
    y_col="Id",
    x_col="Image",
    directory=train_dir,
    target_size = (300,300),
    batch_size=20,
    class_mode="categorical"
)
validation_generator = test_datagen.flow_from_dataframe(
    a,
    y_col="Id",
    x_col = "Image",
    directory=validation_dir,
    target_size = (300,300),
    batch_size=20,
    class_mode="categorical",
)

Found 16728 images belonging to 5005 classes.
Found 4182 images belonging to 5005 classes.


In [154]:
for data, labels in train_generator:
    print(data.shape, labels.shape)
    break

(20, 300, 300, 3) (20, 5005)


## Baseline CNN Model


In [11]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(5005, activation='sigmoid'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 298, 298, 32)      896       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 147, 147, 64)      18496     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 71, 71, 128)       73856     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 33, 33, 128)       147584    
__________

In [13]:
from keras import optimizers
model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

In [ ]:
model.fit_generator()